In [2]:
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse.linalg import svds
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

In [3]:
df = pd.read_json('goodreads_reviews_poetry.json',lines=True)

In [4]:
df = df[['user_id','book_id','rating']]

In [5]:
user_counts = df['user_id'].value_counts()
book_counts = df['book_id'].value_counts()

filtered_users = user_counts[user_counts >= 100].index
filtered_books = book_counts[book_counts >= 100].index

filtered_df = df[df['user_id'].isin(filtered_users)]
filtered_df = df[df['book_id'].isin(filtered_books)]

In [7]:
filtered_df

,user_id,book_id,rating
0,3ca7375dba942a760e53b726c472a7dd,402128,5
6,af157d0205b8a901dee6d4a2aed7e6ad,23534,5
7,f4c6fe33ef61c38f7f4aeb5224c259a5,13105527,5
8,4672eb229c808b792b8ea95f01f19784,1420,1
9,4672eb229c808b792b8ea95f01f19784,1381,3
...,...,...,...
154532,660315d58b3755548cd044eb688914f2,18003300,5
154535,e72f535e284099b2f587712dc5b6ed4f,13376363,5
154543,52d58d3b524bf97067792e273f9ba9fa,1371,5
154549,6bfd86577e73d3d0709451bdc788a501,18003300,5


In [125]:
average_ratings = filtered_df.groupby('book_id')['rating'].mean()


In [129]:
sorted_books = average_ratings.sort_values(ascending=False)

In [ ]:
recommendations = sorted_books.head(n).index.tolist()

In [132]:
rated_items = df[df['user_id'] == "15147f5dde107b2c1fe888f88b28c61b"]['book_id'].tolist()

In [134]:
rated_items

[23513349]

In [146]:
user_ratings = um_poet["15147f5dde107b2c1fe888f88b28c61b"]

In [149]:
df = pd.read_json('/Users/anthonyvillegas/Documents/GitHub/ds-mod-2-0924-code-solutions/recommendation-systems-project/goodreads_reviews_poetry.json',lines=True)
df.to_parquet("poetry.parquet")

In [147]:
num_ratings_above_3 = (user_ratings >= 3).sum()

In [148]:
num_ratings_above_3

np.int64(1)

In [143]:
def recommend_books(user):
    user_ratings = um_poet[user]
    num_ratings_above_3 = (user_ratings >= 3).sum()
    rated_items = df[df['user_id'] == user]['book_id'].tolist()

    if num_ratings_above_3 >= 5: #Only looking at user's that have 5 or more positive ratings
        nn = NearestNeighbors(n_neighbors=4)
        nn.fit(um_corr)
        user_ratings_and_books = {}
        for col in um_poet:
            user_ratings_and_books[col] = um_poet[col].tolist() #This creates a dictionary that we will use as our linear combination to multiply down axis 0 of the correlation matrix to find the best recommendations
        new_matrix = pd.DataFrame((user_ratings_and_books[user] * um_corr).sum(axis=0))
        distance, indices = nn.kneighbors(new_matrix.T)
        poetry_books = um_poet.T.columns[indices[0][1:4]].tolist()
        filtered_recommendations = [item for item in poetry_books if item not in rated_items]
        return poetry_books
    else:  #If they have fewer than 5 positive ratings they are given a generic recommendation list of items with the highest average rating.
        average_ratings = filtered_df.groupby('book_id')['rating'].mean()
        sorted_books = average_ratings.sort_values(ascending=False)
        recommendations = sorted_books.head(3).index.tolist()
        filtered_recommendations = [item for item in recommendations if item not in rated_items]
        return filtered_recommendations

In [144]:
print(recommend_books("15147f5dde107b2c1fe888f88b28c61b"))

[8471387, 13530012, 2168850]


In [140]:
rated_items = df[df['user_id'] == "15147f5dde107b2c1fe888f88b28c61b"]['item_id'].tolist()

KeyError: 'item_id'

In [8]:
filtered_df['book_id'].value_counts()

book_id
23513349    2814
20821284    1048
30075802    1003
1420         984
30119        915
            ... 
333706       101
27822        101
9639765      100
12052640     100
8423931      100
Name: count, Length: 132, dtype: int64

In [9]:
filtered_df['user_id'].value_counts()

user_id
058d86a24d8ed0516eca00476ae21a75    24
7a4b7bb814dc93c18f458a91142d8cb7    20
0017507d4413a03fbfa5848972658206    18
d6bdf3222ea9d58fef73bdeb02ee550a    17
8eab7ab07787a797aa23531baa0a9e31    17
                                    ..
fd9c188688f46074d7d08235ce7d4c8e     1
c7f85472030fd6c7bec2a05286566bb4     1
ede0d1f4c2e530c4bd87ed8286f6e4ef     1
608bb7596b0f1a2b8ed612e833eb12e8     1
0aa3757d0211254712a16bd8289b461d     1
Name: count, Length: 20589, dtype: int64

In [6]:
um_poet = filtered_df.pivot_table(index='book_id',columns='user_id',values='rating').fillna(0)

In [7]:
um_poet2 = filtered_df.pivot_table(index='user_id',columns='book_id',values='rating').fillna(0)


In [8]:
um_corr = um_poet2.corr().fillna(0)

In [9]:
cs_poet = cosine_similarity(um_poet)
cs_poet = pd.DataFrame(cs_poet,columns=um_poet.T.columns,index=um_poet.T.columns)

In [102]:
def create_nn(corr,n):
    nn = NearestNeighbors(n_neighbors=n)
    nn.fit(corr)
    return nn

In [119]:
def recommend_books(user,corr,um,nn_model):
    user_ratings_and_books = {}
    for col in um:
        user_ratings_and_books[col] = um[col].tolist()
    new_matrix = pd.DataFrame((user_ratings_and_books[user] * corr).sum(axis=0))
    distance, indices = nn_model.kneighbors(new_matrix.T)
    poetry_books = um.T.columns[indices[0]].tolist()
    return poetry_books

In [11]:
def svd_values(um,n):
    R = um.values
    um_mean = np.mean(R,axis=1)
    um_mean = um_mean.reshape(-1,1)
    R_demeaned = R - um_mean
    U, sigma, Vt = svds(R_demeaned,k=n)
    sigma = np.diag(sigma)
    return U,sigma,Vt,um_mean

In [12]:
def new_corr_matrix(U,sigma,Vt,um_mean):
    um_reconstructed = U @ sigma @ Vt
    um_reconstructed = um_reconstructed + um_mean
    new_corr = np.corrcoef(um_reconstructed.T)
    new_corr = np.nan_to_num(new_corr)
    return new_corr

In [13]:
def create_nn_svd(new_corr,n):
    nn_svd = NearestNeighbors(n_neighbors=n)
    nn_svd.fit(new_corr)
    return nn_svd

In [14]:
def reco_from_svd(um,user,new_corr,nn_svd):
    ratings_vector = np.array(um.loc[user])
    new_matrix = pd.DataFrame((ratings_vector * new_corr).sum(axis=0))
    distance, indices = nn_svd.kneighbors(new_matrix.T)
    poetry_books = um.columns[indices[0]].tolist()
    return poetry_books

In [15]:
def calc_rmse_matrix(um_poet,um_reconstructed):
    squared_diffs = (um_poet - um_reconstructed)**2
    mean_squared_diffs = np.mean(squared_diffs) 
    rmse = np.sqrt(mean_squared_diffs)
    return rmse


predictions based on test data  for every user in the test data you want to compare those predictions with the 

xtrain is being used to train nn 

stratify split create the same proportions for each column

stratify
array-like, default=None
If not None, data is split in a stratified fashion, using this as the class labels. Read more in the

x train becomes half of the total observations 

creater utility mstricx based on xtrain and corr matrix and then pass corr matrix into nn

for each user you want to take all of the items for each user and filter out so you only get items rated above a certain level.

return the item ids (around 5-10) for the user

test_um and train_corr

compare all k values with original non svd reduced

In [16]:
x_train, x_test = train_test_split(filtered_df, test_size=0.5,train_size=0.5,stratify=filtered_df['book_id'],random_state=1)

In [17]:
train_um = x_train.pivot_table(index='user_id',columns='book_id',values='rating').fillna(0)

In [18]:
test_um = x_test.pivot_table(index='user_id',columns='book_id',values='rating').fillna(0)

In [19]:
def svd_range(um, n_range):
    values = []
    for n in n_range:
        U, sigma, Vt, um_mean = svd_values(um, n)
        values.append((U, sigma, Vt, um_mean))
    return values

In [20]:
values = svd_range(train_um,[10,25,50,100])

In [21]:
def multi_corr_matrices(values):
    corr_matrices = []
    for U, sigma, Vt, um_mean in values:
        corr_matrix = new_corr_matrix(U, sigma, Vt, um_mean)
        corr_matrices.append(corr_matrix)

    return corr_matrices

In [22]:
corr_matrices = multi_corr_matrices(values)

In [23]:
def multi_nn_models(corr_matrices,n):
    nn_models = []
    for matrix in corr_matrices:
        nn_model = NearestNeighbors(n_neighbors=n)
        nn_model.fit(matrix)
        nn_models.append(nn_model)
    return nn_models

In [24]:
nn_models = multi_nn_models(corr_matrices,10)

In [28]:
def reco_multi_svd(um,user,corr,nn_models):
    vector = np.array(um.loc[user])
    new_matrices = []
    for matrix in corr:
        new_matrix = pd.DataFrame((vector * matrix).sum(axis=0))
        new_matrices.append(new_matrix)

    all_poetry_recs = []

    for i,model in enumerate(nn_models):
        distance, indices = model.kneighbors(new_matrices[i].T)
        poetry_books = um.columns[indices[0]].tolist()
        all_poetry_recs.append(poetry_books)

    return all_poetry_recs

In [29]:
all_recs = reco_multi_svd(test_um,"0004ae25e3cf5f5a44b6f1ccfdd3d343",corr_matrices,nn_models)

In [31]:
def get_user_rated_books(user_id, um):
    user_ratings = um.T.loc[user_id]
    rated_books = user_ratings[user_ratings >= 3].index.tolist()
    return rated_books

In [41]:
rated_list = get_user_rated_books("15147f5dde107b2c1fe888f88b28c61b",um_poet)

In [42]:
rated_list

[23513349]

In [1]:
from IPython.display import Markdown, display

In [71]:
def match_finder(og_um,train_um,n_range,test_um):
    values = svd_range(train_um,n_range)
    corr_matrices = multi_corr_matrices(values)
    nn_models = multi_nn_models(corr_matrices,10)
    recs = {}
    column_names = [f"k={i}" for i in n_range]
    counters = pd.DataFrame(columns=column_names,index=test_um.index)
    for user in test_um.index:
        rec = reco_multi_svd(test_um,user,corr_matrices,nn_models)
        recs[user] = rec

    for user in recs:
        rated = set(get_user_rated_books(user, og_um))
        counter_list =[]
        for li in recs[user]:
            counter_list.append(len(set(li).intersection(rated)))
        counters.loc[user] = counter_list
    
    counters_series = counters.sum()
    display(counters_series)

In [120]:
def matches_og(n,um,corr,test_um):
    nn = create_nn(corr,n)
    recs = {}
    column_names = ["matches"]
    counters = pd.DataFrame(columns=column_names,index=test_um.T.index)
    for user in test_um.index:
        rec = recommend_books(user=user,corr=corr,um=test_um.T,nn_model= nn)
        print(rec)
        recs[user] = rec
    for user in recs:
        rated = set(get_user_rated_books(user,um))
        counter_list = []
        counter_list.append(len(set(recs[user]).intersection(rated)))
        counters.loc[user] = counter_list
    counters_series = counters.sum()
    display(counters_series)

In [124]:
matches_og(10,um_poet,um_corr,test_um)

[394424, 58098, 17333426, 99944, 27822, 5932, 1724365, 45974, 519112, 46199]
[291, 2547, 10806415, 9658269, 13499078, 5582346, 3322992, 20765669, 23519886, 6698181]
[10806415, 4488657, 2073853, 6016859, 4096668, 7824768, 5886412, 13499078, 3014384, 11248257]
[18003300, 22151696, 25330489, 23627418, 29431081, 13105527, 34023590, 29801264, 20821097, 18288210]
[6150360, 3018318, 6054132, 2073853, 10691701, 13499078, 5868421, 3322992, 8471387, 11248257]
[34023590, 25986827, 29431081, 23627418, 17333426, 18288210, 25330489, 20821097, 10843755, 29801264]
[11409124, 8423931, 12907847, 8901945, 17707772, 25897953, 20821284, 18585282, 20821097, 17243953]
[23513349, 2547, 25334576, 30075802, 19351, 25986827, 45974, 20613761, 96259, 6798263]


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x10da8a390>>
Traceback (most recent call last):
  File "/Users/anthonyvillegas/Library/Python/3.12/lib/python/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


[8901945, 8411631, 18263725, 20660824, 13530012, 25897953, 12907847, 7493149, 8423931, 17243953]
[7824768, 9780454, 12052640, 9858291, 6054132, 3322992, 6150360, 19480887, 10806415, 16170625]
[19351, 27822, 14706, 34080, 1371, 3049, 15645, 1381, 2696, 15997]
[30118, 12907847, 23919, 519112, 15645, 25986827, 27494, 25334576, 17333426, 400412]
[3018318, 2073853, 6016859, 3014384, 10243089, 8471387, 7787218, 11248257, 6054132, 11296693]
[22151696, 18003300, 25330489, 29431081, 23627418, 34023590, 13105527, 20821097, 23522212, 18288210]
[402128, 732562, 395090, 9639765, 3049, 34080, 42038, 6364068, 29358559, 11904233]
[20483085, 13530012, 12907847, 20660824, 17243953, 14553840, 20821284, 33004289, 16099190, 18263725]
[394424, 17707772, 58098, 676, 29358559, 13588404, 9639765, 333706, 732562, 1724365]
[23919, 27494, 732562, 99944, 15997, 45974, 14553840, 402128, 400412, 82356]
[6343963, 4488657, 16170625, 5868421, 6154130, 8471387, 10810038, 7787218, 6150360, 19480887]
[11904233, 16099190, 

KeyboardInterrupt: 

hello

In [73]:
match_finder(um_poet,train_um,[1,2,3,4],test_um)

k=1     6451
k=2     9886
k=3    10067
k=4    10299
dtype: object

In [84]:
rated = get_user_rated_books("0004ae25e3cf5f5a44b6f1ccfdd3d343",um_poet)

In [86]:
recs_no_svd = recommend_books("0004ae25e3cf5f5a44b6f1ccfdd3d343",10)